In [1]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import json
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Create example bash calls from workflow
Works best with non-restarted WF

In [2]:
project = "d3b-bixu/dev-rnaseq-snv"
task_id = "8b37cb5b-ef56-4f3b-a7c9-87a89b07171f"
out_file = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-06-16_gatk4.tsv", "w")
# task_id = "3c20cc8e-18d7-43f2-bc2c-4a76d38a88f8"
task = api.tasks.get(task_id)
jobs = {}
temp = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED":
        check = job.name.split('_')
        cmd = job.command_line
        if job.command_line == None:
            # pdb.set_trace()
            cmd = "embedded script or task retry"
            sys.stderr.write("WARN: Job " + job.name + " had null cmd\n")
        if check[-1] == "s":
            key = "_".join(check[:-2])
            if key not in temp:
                jobs[job.start_time] = {}
                jobs[job.start_time][key] = cmd
                temp[key] = 1
            else:
                temp[key] += 1
        else:
            jobs[job.start_time] = {}
            jobs[job.start_time][job.name] = cmd
out_file.write("Step\tType\tNum scatter\tCommand\n")
for rtime in sorted(jobs.keys()):
    for key in jobs[rtime]:
        rtype = "run step"
        sct = "NA"
        if key in temp and temp[key] > 1:
            rtype = "scatter"
            sct = str(temp[key])
        cmds = jobs[rtime][key].split('\n')
        for cmd in cmds:
            out_file.write(key + "\t" + rtype + "\t" + sct + "\t" + cmd + "\n")
out_file.close()


## Convert tsv to markdown table

In [3]:
import sys

# max desired col width
max_w = 200
tsv_in = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-06-16_gatk4.tsv")
out_md = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/2020-06-16_gatk4.md", "w")
data = []
max_len = []


for line in tsv_in:
    info = line.rstrip('\n').split('\t')
    data.append(info)
    if len(max_len) == 0:
        for item in info:
            max_len.append(len(item))
    else:
        for i in range(len(max_len)):
            if len(info[i]) > max_w:
                max_len[i] = max_w
            elif len(info[i]) > max_len[i]:
                max_len[i] = len(info[i])
# print header first
d_ct = []
for i in range(len(data[0])):
    d_ct.append(len(data[0][i]))
    out_md.write(" | " + data[0][i] + "".join([" "] * max_len[i]))
    d_ct[i] += max_len[i]
out_md.write(" |\n")
for i in range(len(data[0])):
    out_md.write(" | " + "".join(["-"] * d_ct[i]))
out_md.write(" |\n")
# pdb.set_trace()
for i in range(1, len(data), 1):
    for j in range(len(data[i])):
        d_ct = len(data[i][j]) + 2
        out_md.write(" | " + data[i][j] + "".join([" "] * max_len[j]))
        d_ct += max_len[j]
    out_md.write(" |\n")
out_md.close()

## Get run times

### Get run times by step

In [2]:
def get_job_run_time(task, phrase):
    data = []
    if re.search(phrase, task.name):
        try:
            for job in task.get_execution_details().jobs:
                if job.status != "COMPLETED":
                    sys.stderr.write("Skipping job likely killed due to spot instance kill for " + job.name + " from task " + task.id + "\n")
                else:
                    data.append([job.name, str((job.end_time-job.start_time).seconds/3600)])
            # pdb.set_trace()
            hold=1
            return task.id, task.name, str(task.price.amount), str((task.end_time - task.start_time).seconds/3600), data
        except Exception as e:
            return [e, task.id]
    else:
        return []


In [3]:
project = "kfdrc-harmonization/sd-bhjxbdqk-06"
phrase = "RNAfusion-"

tasks = api.tasks.query(project=project, status="COMPLETED").all()
actual_out = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/actual_cost.txt", "w")
actual_out.write("Task name\tTask ID\tCost\tRun Time in hours\n")
step_run = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/step_run_times.txt", "w")
step_run.write("Run step\tRun time in hours\n")
# for task in tasks:
#     result = get_job_run_time(task, phrase)
#     if len(result) > 0:
#         pdb.set_trace()
#         actual_out.write("\t".join(result[0:4]) + "\n")
#         for step in result[4]:
#             step_run.write("\t".join(step) + "\n")
x = 0
m = 100
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(get_job_run_time, task, phrase): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if len(result.result()) > 2:
            if x % m == 0:
                sys.stderr.write("Processed " + str(x) + " valid tasks\n")
            actual_out.write("\t".join(result.result()[0:4]) + "\n")
            for step in result.result()[4]:
                step_run.write("\t".join(step) + "\n")
            x += 1
        elif len(result.result()) == 2:
            sys.stderr.write(str(result.result()[0]) + "\tFailed processing task ID " + result.result()[1] + "\n")
            exit(1)
actual_out.close()
step_run.close()


Skipping job likely killed due to spot instance kill for star_fusion from task 90520afe-23e8-408e-a6d1-7a4307bff6cb
Skipping job likely killed due to spot instance kill for star_fusion from task ad4166df-eb20-4403-9e62-a434fc023946
Skipping job likely killed due to spot instance kill for pizzly from task b75638c2-fdc0-4321-8320-b7b3b7b6940e
Skipping job likely killed due to spot instance kill for star_fusion from task cf2e69a6-c9b8-47d3-92a6-52c41f4cbc2a
Skipping job likely killed due to spot instance kill for star_fusion from task c011ab43-7a02-4892-981d-5f897de739ac
Skipping job likely killed due to spot instance kill for arriba_fusion from task b6276343-2100-44c2-afc7-23a45ba141cf
Skipping job likely killed due to spot instance kill for pizzly from task c9cff929-f971-41b1-90b0-bf0a3e40f2fb
Skipping job likely killed due to spot instance kill for star_fusion from task 5d9f1b03-f375-4363-bfaf-304fadb3ee6f
Skipping job likely killed due to spot instance kill for star_fusion from task 4

## Tag source files

In [7]:
def tag_file(info, header):
    try:
        meta = info.rstrip('\n').split('\t')
        f_obj = api.files.get(meta[0])
        metadata = {}
        for i in range(3, len(header), 1):
            metadata[header[i]] = meta[i]
        f_obj.metadata = metadata
        f_obj.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
        sys.stderr.write("Could not process " + info)
        exit(1)

In [8]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/covwc_to_tag.txt")
head = next(manifest)
header = head.rstrip("\n").split("\t")
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(tag_file, line, header): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' files\n')
            sys.stderr.flush()


No relevant changes were detected in order to update the resource on the server.
No relevant changes were detected in order to update the resource on the server.
Could not process 5eebb671e4b0a6d31133e684	COVWC-20200312-P2-E02-P.all-reads_Aligned.sortedByCoord.out.bam	d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study	COVWC-20200312-P2-E02-P	Positive
No relevant changes were detected in order to update the resource on the server.
Could not process 5eebb672e4b0a6d31133e6a7	COVWC-20200312-P2-E02-P.all-reads_ReadsPerGene.out.tab	d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study	COVWC-20200312-P2-E02-P	Positive
No relevant changes were detected in order to update the resource on the server.
Could not process 5eebb672e4b0a6d31133e693	COVWC-20200312-P2-E02-P.all-reads_Chimeric.out.junction	d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study	COVWC-20200312-P2-E02-P	Positive
No relevant changes were detected in order to update the resource on the server.
Could not process 5eebb673e4b0a6d

No relevant changes were detected in order to update the resource on the server.
Could not process 5eebb67ae4b0a6d31133e7d1	COVWC-20200313-P4-G01-N.all-reads_ReadsPerGene.out.tab	d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study	COVWC-20200313-P4-G01-N	Negative


## Set up GATK4 RNAseq WF Tasks

In [12]:
def get_gatk_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['reference_fasta'] = api.files.get('5eebc5d1e4b0a6d311357eb9')
        ref_dict['reference_dict'] = api.files.get('5eecb14ae4b0efd899f474da')
        known_sites = []
        known_sites.append(api.files.get('5eecd8c3e4b0efd899f4ae44'))
        known_sites.append(api.files.get('5eecd846e4b0efd899f4ae27'))
        known_sites.append(api.files.get('5eecd846e4b0efd899f4ae26'))
        known_sites.append(api.files.get('5eecd846e4b0efd899f4ae21'))
        ref_dict['knownsites'] = known_sites
        ref_dict['call_bed_file'] = api.files.get('5eebdeece4b0efd899f43eaa')
        ref_dict['dbsnp_vcf'] = api.files.get('5eecd846e4b0efd899f4ae24')
        ref_dict['tool_name'] = 'STAR_GATK4'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [13]:
def draft_task(in_file):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        info = in_file.rstrip('\n').split('\t')
        input_dict['STAR_sorted_genomic_bam'] = api.files.get(info[0])
        task_name = "GATK RNAseq SNV: " + info[3]
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [15]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/d3b-gatk-rnaseq-snv-wf"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/bams_for_gatk_to_run.tsv")
head = next(manifest)
ref_obj = get_gatk_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_task, line ): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()


## Run VEP

In [12]:
def get_vep_refs(api, project):
    try:
        ref_dict = {}
        ref_dict['reference'] = api.files.get('5eebc5d1e4b0a6d311357eb9')
        ref_dict['cache'] = api.files.get('5eed0f54e4b0efd899f4afda')
        ref_dict['merged_cache'] = True
        ref_dict['tool_name'] = 'STAR_GATK4'
    except Exception as e:
        sys.stderr.write(str(e) + "\nFailed to get REFS\n")
        exit(1)
    return ref_dict

In [13]:
def draft_vep_task(in_file):
    try:
        input_dict = {}
        for key in ref_obj:
            input_dict[key] = ref_obj[key]
        info = in_file.rstrip('\n').split(',')
        input_dict['input_vcf'] = api.files.get(info[0])
        task_name = "VEP R100 ANNOTATE: " + info[3]
        task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
        task.inputs['output_basename'] = task.id
        task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nfailed to set up task for " + in_file)
        exit(1)

In [15]:
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'
app_name = project + "/vep-1oo-annotate"
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/vcf_to_annotate-manifest.csv")
head = next(manifest)
ref_obj = get_vep_refs(api, project)
x = 1
m = 250
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(draft_vep_task, line ): line for line in manifest}
    for result in concurrent.futures.as_completed(results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks\n')
            sys.stderr.flush()


## Copy metadata to outputs

In [2]:
def add_metadata_to_outputs(task, phrase, in_key):
    if re.search(phrase, task.name):
        sys.stderr.write('Valid task found ' + task.name + '\n')
        metadata = task.inputs[in_key].metadata
        for out_key in task.outputs:
            # pdb.set_trace()
            try:
                if type(task.outputs[out_key]) is not list:
                    file_obj = api.files.get(task.outputs[out_key].id)
                    for key in metadata:
                        file_obj.metadata[key] = metadata[key]
                    file_obj.save()
                else:
                    for output in task.outputs[out_key]:
                        if type(output) is not list:
                            file_obj = api.files.get(output.id)
                            for key in metadata:
                                file_obj.metadata[key] = metadata[key]
                            file_obj.save()
                        else:
                            for item in output:
                                if item is not None:
                                    file_obj = api.files.get(item.id)
                                    for key in metadata:
                                        file_obj.metadata[key] = metadata[key]
                                    file_obj.save()

            except Exception as e:
                print(e)
                print("Skipping " + out_key + " for " + task.name + " due to error")

In [16]:
prefix = 'VEP R100 ANNOTATE'
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
key = 'input_vcf'
print("You sure tag outputs with task prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":
    tasks = api.tasks.query(project=project, status="COMPLETED").all()
    for task in tasks:
        add_metadata_to_outputs(task, prefix, key)
else:
    sys.stderr.write("User did not type YASS, skipping\n")

You sure tag outputs with task prefix: VEP R100 ANNOTATE? Type "YASS" if so
YASS


Valid task found VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP


No relevant changes were detected in order to update the resource on the server.
Skipping output_txt for VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP due to error
No relevant changes were detected in order to update the resource on the server.
Skipping output_vcf for VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP due to error
No relevant changes were detected in order to update the resource on the server.
Skipping warn_txt for VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP due to error


Valid task found VEP R100 ANNOTATE: COVHA-20200312-P3-F07-N
Valid task found VEP R100 ANNOTATE: COVHA-20200312-P2-D02-P


'NoneType' object has no attribute 'id'
Skipping warn_txt for VEP R100 ANNOTATE: COVHA-20200312-P3-F07-N due to error


...

Valid task found VEP R100 ANNOTATE: COVHA-20200403-P2-E04-N


'NoneType' object has no attribute 'id'
Skipping warn_txt for VEP R100 ANNOTATE: COVHA-20200403-P2-E04-N due to error


Valid task found VEP R100 ANNOTATE: COVHA-20200314-P7-C08-P
Valid task found VEP R100 ANNOTATE: COVHA-20200403-P2-G05-N
Valid task found VEP R100 ANNOTATE: COVHA-20200316-P11-G07-N


'NoneType' object has no attribute 'id'
Skipping warn_txt for VEP R100 ANNOTATE: COVHA-20200403-P2-G05-N due to error


Valid task found VEP R100 ANNOTATE: COVHA-20200314-P8-G01-P


## Get task outputs

In [2]:
def write_to_manifest(out_fh, file_obj, out_key, task_name):
    out_fh.write(",".join([file_obj.id, file_obj.name, out_key, task_name]) + "\n")

In [5]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
out = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/test_vep_out.txt', 'w')
out.write("id,name,output_category,task_name\n")
phrase = "VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP"

for task in tasks:
    if re.search(phrase, task.name):
        sys.stderr.write('Processing task: ' + task.name + "\n")
        for out_key in task.outputs:
#             try:
            if type(task.outputs[out_key]) is not list:
                file_obj = task.outputs[out_key]
                write_to_manifest(out, file_obj, out_key, task.name)
                if task.outputs[out_key].secondary_files is not None:
                    write_to_manifest(out, task.outputs[out_key].secondary_files[0], out_key, task.name)
            else:
                for i in range(len(task.outputs[out_key])):
                    if type(task.outputs[out_key][i]) is not list:
                        write_to_manifest(out, task.outputs[out_key][i], out_key, task.name)
                        if task.outputs[out_key][i].secondary_files is not None:
                            write_to_manifest(out, task.outputs[out_key][i].secondary_files[0], out_key, task.name)
                    else:
                        for j in range(len(task.outputs[out_key][i])):
                            if task.outputs[out_key][i][j] is not None:
                                write_to_manifest(out, task.outputs[out_key][i][j], out_key, task.name)
                                if task.outputs[out_key][i][j].secondary_files is not None:
                                    write_to_manifest(out, task.outputs[out_key][i][j].secondary_files[0], out_key, task.name)
#             except Exception as e:
#                 print(e)
#                 print("Skipping " + out_key + " for " + task.name + " due to error")
out.close()


Processing task: VEP R100 ANNOTATE: COVHA-20200311-P1-A01-CP


## Abort unresponsive and restart tasks

In [23]:
import datetime
import pytz

project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="RUNNING")
current = datetime.datetime.now()
tz = pytz.timezone('America/New_York')
prefix = "GATK RNAseq SNV"
task_abort = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/aborted_and_restarted2.log", 'w')
task_abort.write("Task ID\tTask name\tNew Task ID")
for task in tasks:
    if re.search(prefix, task.name):
        for job in task.get_execution_details().jobs:
            if job.name == "preprocess_rnaseq_bam_sambamba_md_sorted":
                if job.status == "RUNNING":
                    diff = (current-pytz.utc.localize(job.start_time, is_dst=None).astimezone(tz).replace(tzinfo=None)).seconds/3600
                    if diff > 2:
                        task_abort.write(task.id + "\t" + task.name)
                        in_dict = {}
                        sys.stderr.write("Aborting " + task.id + "\t" + task.name + "\n" )
                        task.abort()
                        new_task = task.clone(run=False)
                        new_task.inputs['output_basename'] = new_task.id
                        new_task.save()
                        task_abort.write("\t" + new_task.id + "\n")
                        task_abort.flush()
                    else:
                        break
                else:
                    break
task_abort.close()

Aborting 2b233b53-4e04-4691-ba01-114e806a27b6	GATK RNAseq SNV: COVHA-20200403-P2-B06-N
Aborting 26c0855b-3116-48fa-9f6f-680b3eadd3ef	GATK RNAseq SNV: COVHA-20200403-P2-E04-N
Aborting 44d400a2-4d1f-464e-93cf-32668e5da436	GATK RNAseq SNV: COVHA-20200314-P7-C08-P


## Restart failed tasks

In [20]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="FAILED")
prefix = "GATK RNAseq SNV"
task_restart = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/failed_and_restarted2.log", 'w')
task_restart.write("Task ID\tTask name\tNew Task ID\n")
run_list = ["GATK RNAseq SNV: COVHA-20200315-P9-F02-N",
             "GATK RNAseq SNV: COVHA-20200316-P12-F01-P","GATK RNAseq SNV: COVHA-20200403-P1-C06-P"]
for task in tasks:
    if re.search(prefix, task.name) and task.name in run_list:
        task_restart.write(task.id + "\t" + task.name)
        new_task = task.clone(run=False)
        new_task.inputs['output_basename'] = new_task.id
        new_task.save()
        task_restart.write("\t" + new_task.id + "\n")
        task_restart.flush()
task_restart.close()


### Get Failed list

In [18]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="FAILED")
prefix = "GATK RNAseq SNV"
task_failed = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/TASK_RUN/failed.log", 'w')
task_failed.write("Task ID\tTask name\n")
for task in tasks:
    if re.search(prefix, task.name):
        task_failed.write(task.id + "\t" + task.name + "\n")
task_failed.close()

### Remove outputs from failed and aborted tasks

In [9]:
project = "d3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study"
tasks = api.tasks.query(project=project, status="ABORTED")
prefix = "GATK RNAseq SNV"
print("You sure remove outputs from failed tasks with prefix: " + prefix + "? Type \"YASS\" if so")
check = input()
if check == "YASS":

    for task in tasks:
        if re.search(prefix, task.name):
            for key in task.outputs:
                if task.outputs[key] is not None:
                    sys.stderr.write("Found files to remove from failed task: " + task.id + " " + task.name + "\n")
                    try:
                        if task.outputs[key].secondary_files is not None:
                            sys.stderr.write("Removing secondary files\n")
                            for i in range(0, len(task.outputs[key].secondary_files), 1):
                                task.outputs[key].secondary_files[i].delete()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nFile with key " + key + " probably does not have secondaryFiles, skipping\n")
                    try:
                        task.outputs[key].delete()
                    except Exception as e:
                        sys.stderr.write(str(e) + "\nFile with key " + key + " was probably deleted before, skipping\n")            
                    sys.stderr.write("Finished processing " + task.id + "\n")


You sure remove outputs from failed tasks with prefix: GATK RNAseq SNV? Type "YASS" if so
YASS


### Rename _\d_ files

In [11]:
manifest = open("/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/manifests/to_rename-manifest.csv")
head = next(manifest)
print("You sure you want to rename the files in that manifest? Type \"YASS\" if so")
check = input()
if check == "YASS":
    for line in manifest:
        info = line.split(',')
        cur = api.files.get(info[0])
        new_name = cur.name[3:]
        sys.stderr.write("Renaming file with ID " + cur.id + " " + cur.name + " to " + new_name + "\n")
        cur.name = new_name
        cur.save()

You sure you want to rename the files in that manifest? Type "YASS" if so
YASS


Renaming file with ID 5eed644ae4b03703de625f64 _1_COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.recal_data.csv to COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.recal_data.csv
Renaming file with ID 5eed644ae4b03703de625fd9 _1_COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.aligned.duplicates_marked.recalibrated.bam to COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.aligned.duplicates_marked.recalibrated.bam
Renaming file with ID 5eed644ae4b03703de625fe1 _1_COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.aligned.duplicates_marked.recalibrated.bam.md5 to COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.aligned.duplicates_marked.recalibrated.bam.md5
Renaming file with ID 5eed644ae4b03703de625fe3 _1_COVHA-20200314-P7-C07-P.all-reads_Aligned.sortedByCoord.out.md.splitn.aligned.duplicates_marked.recalibrated.bai to COVHA-20200314-P7-C07-P.all-reads_Aligned.sor